In [7]:
import tensorflow as tf
from tensorflow.keras import layers,Model
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D
from tensorflow.keras.applications.resnet_v2 import ResNet50V2
from scipy.io import loadmat
import cv2
import numpy as np

from config import Config

config = Config()


framess = np.load("/data/lcasini/Dataset/run2/132_46/frames_132_46.npy")

#primo frame
image = framess[0]
#immagine convertita in float
image = tf.image.convert_image_dtype(image, dtype=tf.float32)
image_size = tf.cast(tf.shape(image)[:2], tf.float32)

#resize dell'immagine
encoder_img_size = config.ENCODER_INPUT_SHAPE[:2]
image_resize = tf.image.resize(image, encoder_img_size, method=tf.image.ResizeMethod.NEAREST_NEIGHBOR)

# Normalize image to [-1, 1]
image_final = tf.subtract(image_resize, 0.5)
image_final = tf.multiply(image_final, 2.0)

enc_shape = config.ENCODER_INPUT_SHAPE

#inizializzata la resnet
resnet50V2 = ResNet50V2(include_top=False, weights='imagenet', input_shape=enc_shape, pooling='avg')
vs_initializer = tf.keras.initializers.VarianceScaling(2.0)

l2_regularizer = tf.keras.regularizers.l2(0.0010000000474974513)#non so se il valore è corretto

for layer in resnet50V2.layers:
            if isinstance(layer, layers.Conv2D):
                # original implementations slim `resnet_arg_scope` additionally sets
                # `normalizer_fn` and `normalizer_params` which in TF 2.0 need to be implemented
                # as own layers. This is not possible using keras ResNet50V2 application.
                # Nevertheless this is not needed as training seems to be likely stable.
                # See https://www.tensorflow.org/guide/migrate#a_note_on_slim_contriblayers for more
                # migration insights
                setattr(layer, 'padding', 'same')
                setattr(layer, 'kernel_initializer', vs_initializer)
                setattr(layer, 'kernel_regularizer', l2_regularizer)
            if isinstance(layer, layers.BatchNormalization):
                setattr(layer, 'momentum', 0.997)
                setattr(layer, 'epsilon', 1e-5)
            if isinstance(layer, layers.MaxPooling2D):
                setattr(layer, 'padding', 'same')

if len(tf.shape(image_final)) != 4:
            image_final = tf.expand_dims(image_final, 0)
features = resnet50V2(inputs=image_final, training=False)
print(features)



tf.Tensor([[0.16366288 0.05877373 0.         ... 0.17657517 0.01568415 0.05696955]], shape=(1, 2048), dtype=float32)
